In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# ----------------------
# 1. Generate toy dataset: points on a circle
# ----------------------
n = 5000
theta = 2 * np.pi * np.random.rand(n)
x = np.c_[np.cos(theta), np.sin(theta)]  # (n, 2)

X_train, X_test = train_test_split(x, test_size=0.2)

# ----------------------
# 2. Define Autoencoder
# ----------------------
latent_dim = 2

encoder = models.Sequential([
    layers.Input(shape=(2,)),
    layers.Dense(16, activation="relu"),
    layers.Dense(latent_dim)  # latent code
])

decoder = models.Sequential([
    layers.Input(shape=(latent_dim,)),
    layers.Dense(16, activation="relu"),
    layers.Dense(2)  # reconstruct original 2D point
])

inp = layers.Input(shape=(2,))
code = encoder(inp)
recon = decoder(code)
ae = models.Model(inp, recon)

ae.compile(optimizer="adam", loss="mse")
ae.fit(X_train, X_train, epochs=50, batch_size=64, verbose=0)

# ----------------------
# 3. Visualize latent space of AE
# ----------------------
z_train = encoder.predict(X_train)
z_test = encoder.predict(X_test)

plt.figure(figsize=(12,5))

# Plot latent codes of training data
plt.subplot(1,2,1)
plt.scatter(z_train[:,0], z_train[:,1], alpha=0.5, s=5)
plt.title("Autoencoder latent space")
plt.xlabel("z1"); plt.ylabel("z2")

# Plot reconstructions from a grid in latent space
plt.subplot(1,2,2)
grid_x, grid_y = np.meshgrid(np.linspace(-3,3,20), np.linspace(-3,3,20))
grid = np.c_[grid_x.ravel(), grid_y.ravel()]
recon_points = decoder.predict(grid)

plt.scatter(X_train[:,0], X_train[:,1], alpha=0.2, label="Training data")
plt.scatter(recon_points[:,0], recon_points[:,1], c="red", s=10, label="Reconstructions")
plt.legend()
plt.title("Decoder outputs from latent grid")
plt.axis("equal")
plt.show()
